In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Preparing Data

In [2]:
def append_bigrams(x):
    bigrams = set(["%s %s" % (f, s) for f, s in zip(x[:-1], x[1:])])
    x.extend(list(bigrams))
    return x

append_bigrams("This film is the best".split())

['This',
 'film',
 'is',
 'the',
 'best',
 'the best',
 'film is',
 'This film',
 'is the']

In [3]:
import torchtext
from torchtext.data import Field, LabelField, BucketIterator

TEXT = Field(tokenize='spacy', tokenizer_language='en_core_web_sm', preprocessing=append_bigrams, include_lengths=True)
LABEL = LabelField(dtype=torch.float)

train_data, test_data = torchtext.datasets.IMDB.splits(TEXT, LABEL, root="../assets/data")
train_data, valid_data = train_data.split()

In [4]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE, 
                 vectors="glove.6B.100d", vectors_cache="../assets/vector_cache", 
                 unk_init=torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [5]:
# The bi-grams are included in the `Dataset`.
print(train_data[0].text)
# The bi-grams are also included in the `TEXT.vocab`.
print(TEXT.vocab.itos[-10:])

['There', 'is', 'a', 'story', '(', 'possibly', 'apocryphal', ')', 'about', 'an', 'exchange', 'between', 'Bruce', 'Willis', 'and', 'Terry', 'Gilliam', 'at', 'the', 'start', 'of', 'Twelve', 'Monkeys', '.', 'Gilliam', '(', 'allegedly', ')', 'produced', 'a', 'long', 'list', '(', 'think', 'about', 'the', 'aircraft', 'one', 'from', 'the', 'Fifth', 'Element', ')', 'and', 'handed', 'it', 'to', 'Butch', 'Bruce', '.', 'It', 'was', 'entitled', '"', 'Things', 'Bruce', 'Willis', 'Does', 'When', 'He', 'Acts', '"', '.', 'It', 'ended', 'with', 'a', 'simple', 'message', 'saying', ':', '"', 'please', 'do', "n't", 'do', 'any', 'of', 'the', 'above', 'in', 'my', 'movie".<br', '/><br', '/>There', 'is', 'a', 'fact', 'about', 'this', 'movie', '(', 'definitely', 'true', ')', '.', 'Gilliam', 'did', "n't", 'have', 'a', 'hand', 'in', 'the', 'writing.<br', '/><br', '/>I', 'would', 'contend', 'that', 'these', 'two', 'factors', 'played', 'a', 'huge', 'role', 'in', 'creating', 'the', 'extraordinary', '(', 'if', 'not'

In [6]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, device=device)

In [7]:
for batch in train_iterator:
    break
text, text_lens = batch.text
print(text)
print(text_lens)

tensor([[  452, 11558,    11,  ..., 20269,    56,  1003],
        [  111,    67,    19,  ...,     3,    15,    48],
        [   83,     2,  3042,  ...,    16,   199,    13],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]], device='cuda:0')
tensor([ 457,  864,  503, 1190, 1293,  512,  989,  330,  309,  842, 1298, 1140,
         192,  291,  588,  256,  572,  568,  379,  327, 1466,  430,  402,  460,
         494,  515,  347, 1174,  107,  342, 2459,  358,  154,  297,  516,  741,
         198,  407,  171,  234,  278, 1273,  855,  777,  657,  420,  539,  573,
         344,  506,  224,  231,  309,  309,  460, 1182,  221,  408,  179,  677,
         341,  660,  274,  240], device='cuda:0')


# Build Model

In [8]:
class Classifier(nn.Module):
    def __init__(self, in_dim, emb_dim, out_dim, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        self.fc = nn.Linear(emb_dim, out_dim)

    def forward(self, text, text_lens):
        # text: (step, batch)
        embedded = self.emb(text)
        
        # Pooling along steps
        # pooled: (batch, emb)
        pooled = embedded.sum(dim=0) / text_lens.unsqueeze(1)
        return self.fc(pooled)

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

IN_DIM = len(TEXT.vocab)
EMB_DIM = 100
OUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

classifier = Classifier(IN_DIM, EMB_DIM, OUT_DIM, PAD_IDX).to(device)
count_parameters(classifier)

2500301

In [10]:
# Initialize Embeddings with Pre-Trained Vectors
classifier.emb.weight.data.copy_(TEXT.vocab.vectors)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

classifier.emb.weight.data[UNK_IDX].zero_()
classifier.emb.weight.data[PAD_IDX].zero_()

print(classifier.emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344],
        [-0.1077,  0.1105,  0.5981, -0.5436,  0.6740,  0.1066,  0.0389,  0.3548],
        [-0.3398,  0.2094,  0.4635, -0.6479, -0.3838,  0.0380,  0.1713,  0.1598]],
       device='cuda:0', grad_fn=<SliceBackward>)


# Train Model

In [11]:
optimizer = optim.Adam(classifier.parameters())
# Binary cross entropy with logits. 
# The binary version of cross entropy loss. 
loss_func = nn.BCEWithLogitsLoss().to(device)

In [12]:
def train_epoch(classifier, iterator, optimizer, loss_func):
    classifier.train()
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        # Forward pass
        text, text_lens = batch.text
        preds = classifier(text, text_lens).squeeze(-1)
        # Calculate loss
        loss = loss_func(preds, batch.label)
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        # Update weights
        optimizer.step()
        # Accumulate loss and acc
        epoch_loss += loss.item()
        epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def eval_epoch(classifier, iterator, loss_func):
    classifier.eval()
    epoch_loss = 0
    epoch_acc = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            text, text_lens = batch.text
            preds = classifier(text, text_lens).squeeze(-1)
            # Calculate loss
            loss = loss_func(preds, batch.label)
            # Accumulate loss and acc
            epoch_loss += loss.item()
            epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [13]:
import time
N_EPOCHS = 10
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss, train_acc = train_epoch(classifier, train_iterator, optimizer, loss_func)
    valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = int(epoch_secs // 60), int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(classifier.state_dict(), "models/tut3-model.pt")
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 8s
	Train Loss: 0.670 | Train Acc: 67.51%
	 Val. Loss: 0.629 |  Val. Acc: 76.70%
Epoch: 02 | Epoch Time: 0m 7s
	Train Loss: 0.553 | Train Acc: 80.55%
	 Val. Loss: 0.484 |  Val. Acc: 83.57%
Epoch: 03 | Epoch Time: 0m 7s
	Train Loss: 0.413 | Train Acc: 86.61%
	 Val. Loss: 0.386 |  Val. Acc: 86.27%
Epoch: 04 | Epoch Time: 0m 7s
	Train Loss: 0.327 | Train Acc: 89.30%
	 Val. Loss: 0.335 |  Val. Acc: 87.45%
Epoch: 05 | Epoch Time: 0m 7s
	Train Loss: 0.274 | Train Acc: 90.97%
	 Val. Loss: 0.308 |  Val. Acc: 87.72%
Epoch: 06 | Epoch Time: 0m 7s
	Train Loss: 0.237 | Train Acc: 92.23%
	 Val. Loss: 0.289 |  Val. Acc: 88.66%
Epoch: 07 | Epoch Time: 0m 7s
	Train Loss: 0.208 | Train Acc: 93.27%
	 Val. Loss: 0.278 |  Val. Acc: 89.11%
Epoch: 08 | Epoch Time: 0m 7s
	Train Loss: 0.185 | Train Acc: 94.07%
	 Val. Loss: 0.274 |  Val. Acc: 88.73%
Epoch: 09 | Epoch Time: 0m 7s
	Train Loss: 0.165 | Train Acc: 94.80%
	 Val. Loss: 0.266 |  Val. Acc: 89.16%
Epoch: 10 | Epoch Time: 0m 7

In [14]:
classifier.load_state_dict(torch.load("models/tut3-model.pt"))

valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
test_loss, test_acc = eval_epoch(classifier, test_iterator, loss_func)

print(f'Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Val. Loss: 0.262 | Val. Acc: 89.28%
Test Loss: 0.267 | Test Acc: 89.20%


# User Input

In [15]:
import spacy
nlp = spacy.load('en')

def predict_senti(classifier, sentence):
    classifier.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[tok] for tok in tokenized]
    lens = len(indexed)

    indexed = torch.tensor(indexed, dtype=torch.long).unsqueeze(1).to(device)
    lens = torch.tensor(lens, dtype=torch.long).unsqueeze(0).to(device)
    pred = torch.sigmoid(classifier(indexed, lens)).round().type(torch.long)
    return LABEL.vocab.itos[pred.item()]

In [16]:
predict_senti(classifier, "This is a good film.")

'pos'

In [17]:
predict_senti(classifier, "This film is terrible.")

'neg'

# Check Embeddings
* The Embeddings of `<unk>` and `<pad>` tokens
    * Because the `padding_idx` has been passed to `nn.Embedding`, so the `<pad>` embedding will remain zeros throughout training.  
    * While the `<unk>` embedding will be learned.

In [18]:
print(classifier.emb.weight[:5, :8])

tensor([[-0.0193,  0.0180, -0.0278, -0.0152, -0.0125,  0.0173, -0.0228, -0.0650],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0386, -0.3228,  0.7869, -0.3218,  0.1682, -0.0512, -0.3133,  0.3825],
        [-0.0197,  0.0214,  0.6639, -0.4545,  0.7706,  0.0123,  0.1291,  0.4184],
        [-0.3219,  0.1918,  0.4686, -0.6305, -0.3675, -0.0318,  0.1917,  0.1707]],
       device='cuda:0', grad_fn=<SliceBackward>)
